I plan to update the model in a timely manner. 
For the current situation, I referred to the following.

# if it is useful for you, please vote

* https://www.kaggle.com/code/realneuralnetwork/jpx-lgbm-model-overfitting-high-score
* https://www.kaggle.com/code/paulorzp/median-model-jpx
* https://www.kaggle.com/code/realneuralnetwork/jpx-lgbm-model-overfitting-high-score

# Needs suppression as it is likely to be overfitted.

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")

In [ ]:
NDAYS = 34
lastdays = prices[prices["Date"]>=prices.Date.iat[-2000*NDAYS]].reset_index(drop=True)

In [ ]:
# cols = ['SecuritiesCode', 'Target']
# df = pd.DataFrame(index=[], columns=cols)

# for i,m in lastdays.groupby("SecuritiesCode"):
#     res = float(m["Target"].rolling(34).mean().tail(1))
#     record = pd.Series([int(i), res], index=df.columns)
#     df = df.append(record, ignore_index=True)

In [ ]:
# pred = df
# pred0 = df["Target"].rank(ascending=False,method="first").astype(int) -1
# pred["Target"] = pred0

In [ ]:
# pred["SecuritiesCode"] = pred["SecuritiesCode"].astype("int")

In [ ]:
# pred = dict(zip(pred["SecuritiesCode"],pred["Target"]))

In [ ]:
lastdays = pd.DataFrame(prices.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return lastdays.loc[_id_]
prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)

In [ ]:
prices.Date = pd.to_datetime(prices.Date)
prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)
X=prices[["Date","SecuritiesCode","Avg"]]
y=prices[["Target"]]
codes = X.SecuritiesCode.unique()

In [ ]:
import optuna

def objectives(trial):
    # optunaでのハイパーパラメータサーチ範囲の設定
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 100, 2000),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_bin': trial.suggest_int('max_bin', 2, 100),
            'learning_rate': trial.suggest_uniform('learning_rate',0, 1),
    }

    model = LGBMRegressor(**params)
    model.fit(X,y)
    score = model.score(X,y)
    print(score)
    return score

In [ ]:
opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=0))
opt.optimize(objectives, n_trials=20)

# 最適パラメータ取得
trial = opt.best_trial
params_best = dict(trial.params.items())
params_best['random_seed'] = 0
    
# 最適パラメータで学習/予測    
model_o = LGBMRegressor(**params_best)#

In [ ]:
model_o.fit(X,y)

In [ ]:
# model=LGBMRegressor(num_leaves=500, learning_rate=0.05, n_estimators=100)
# model=LGBMRegressor(num_leaves=2000, learning_rate=0.1, n_estimators=200, max_depth=-1)
# model.fit(X,y)
# model.score(X,y)

In [ ]:
# model

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    sample_prediction["Avg"] = sample_prediction["SecuritiesCode"].apply(get_avg)
    df = sample_prediction[["Date","SecuritiesCode","Avg"]]
    df.Date = pd.to_datetime(df.Date)
    df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    sample_prediction["Prediction"] = model_o.predict(df)
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)

In [ ]:
# env = jpx_tokyo_market_prediction.make_env()
# iter_test = env.iter_test()

# for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
#     sample_prediction['Rank'] = sample_prediction["SecuritiesCode"].map(pred)
#     env.predict(sample_prediction)